In [1]:
import pandas as pd
import numpy as np
import scipy
import csv
import re
import nltk
from nltk.tokenize import TweetTokenizer 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import itertools
from random import *
import time

import matplotlib.pyplot as plt
import matplotlib


pd.set_option('display.max_colwidth', -1) # to show all the content of the colum text

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.preprocessing import StandardScaler 
from sklearn import metrics
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


# import for visualization

%matplotlib inline

try:
    import seaborn as sns
except:
    print('Seaborn must be installed to continue (pip install seaborn).\nIt provides an enhanced plotting experience', file=sys.stderr)
    if input('Do you want me to do it for you? y/n') == 'y':
        !pip install seaborn
    
    import seaborn as sns

# IMPORT TWEETS

In [2]:
#original DB
df = pd.read_csv("C:/Users/cvalentino/Desktop/UB/Project/data/tweets_publics_2.csv", encoding='utf-16', 
                 index_col='tweet_id', 
                 sep=',')

df.airline_sentiment = pd.Categorical(df.airline_sentiment)
df.tweet_created = pd.to_datetime(df.tweet_created)


#cleansed DB

df_clean = pd.read_csv("C:/Users/cvalentino/Desktop/UB/Project/data/tweets_publics_clean.csv", encoding='ansi', 
                 index_col='tweet_id', 
                 sep='\t')


In [3]:
df.head()

,airline_sentiment,is_reply,reply_count,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
tweet_id,,,,,,,,,
942743012337123328,positive,False,0,0,"""Los pilotos de Ryanair desconvocan la huelga tras ver reconocidos sus sindicatos"" by El País via Últimas noticias… https://t.co/80Fz6dxP9t",NaN,2017-12-18 13:07:04,NaN,Dublin
926857871916183553,positive,True,0,0,"@Iberia @lavecinarubia Si ,por favor las declaraciones de amor entre los #rubijarena no pueden quedarse en una ma… https://t.co/GWKJGhhubY",NaN,2017-11-04 17:05:11,NaN,NaN
936979305720090626,neutral,True,0,0,@Iberia Me dirías por favor que costo tiene?,NaN,2017-12-02 15:24:09,NaN,NaN
943983853802328064,negative,True,0,0,"@SupermanlopezN @Iberia @giroditalia Champion, no vuele más con esos descarados.",NaN,2017-12-21 23:17:43,NaN,Central Time (US & Canada)
938207464457211904,negative,True,0,0,@SrtaFarrellDM @KLM @Iberia Eso de avianca es verdad a mi mama le paso dos veces terrible esa aerolínea 😤,NaN,2017-12-06 00:44:25,NaN,Eastern Time (US & Canada)


# EXTRACTION OF EXTENDED TEXT

In [4]:
# TWITTER API SET UP
import tweepy


CONSUMER_KEY = "**"
CONSUMER_SECRET = "**"
OAUTH_TOKEN = "**"
OAUTH_TOKEN_SECRET = "**"


auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
api = tweepy.API(auth)



In [5]:
# RETRIVE TWIT IN SHORT MODE (as-is)

id_of_tweet = 932626561966247936

tweet = api.get_status(id_of_tweet)
print(tweet.text)

No solo en los periódicos encuentro a @JuanitoLibritos , comprando en Ryanair también lo veo como parte de una fami… https://t.co/sSEvHM3h7h


In [6]:
# RETRIVE TWIT IN EXTENDED MODE (TO-BE)

id_of_tweet = 932626561966247936

tweet = (api.get_status(id_of_tweet, tweet_mode='extended')._json['full_text'])
print(tweet)

No solo en los periódicos encuentro a @JuanitoLibritos , comprando en Ryanair también lo veo como parte de una familia estándar... ,,😂😂😂 https://t.co/UqE37UNlhU


In [7]:
#creating a column = to the index to be used as function argument

df['index1'] = df.index

In [8]:
# function to retrieve the whole tweet via tweepy

def tweet_extend (tweet_id):
    
    try:
        return api.get_status(tweet_id, tweet_mode='extended')._json['full_text']
    except:
        return "ERROR"


In [62]:
# NOOOO OOOLD

def tweet_extend (tweet_id):
    
    return api.get_status(tweet_id, tweet_mode='extended')._json['full_text']

In [60]:
# NOOOOOO BETTER NOT TO USE

def tweet_extend (tweet_id):
    
    try:
        return api.get_status(tweet_id, tweet_mode='extended')._json['full_text']
    except:
        time.sleep(15 * 60)
        return "ERROR"
        


In [21]:
# apply the tweet extend function

# prepared for run

df['text_extended'] = df.index1[898:1061].apply(tweet_extend)


# mancano 900 - 1058




In [22]:
# extract extended tweets DB to csv

df.to_csv("C:/Users/cvalentino/Desktop/UB/Project/data/tweets_publics_ext.csv", sep=',')

In [24]:
#re - upload complete file (processed in excel)

df1 = pd.read_csv("C:/Users/cvalentino/Desktop/UB/Project/data/tweets_publics_ext_all.csv", encoding='ANSI', 
                 index_col='tweet_id', 
                 sep=',')




In [27]:
df1.head(5)

,airline_sentiment,is_reply,reply_count,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,index1,text_extended,text_extended_final
tweet_id,,,,,,,,,,,,
9.430000e+17,positive,False,0,0,"""Los pilotos de Ryanair desconvocan la huelga tras ver reconocidos sus sindicatos"" by El País via Últimas noticias… https://t.co/80Fz6dxP9t",NaN,43087.54653,NaN,Dublin,9.430000e+17,"""Los pilotos de Ryanair desconvocan la huelga tras ver reconocidos sus sindicatos"" by El País via Últimas noticias | EL PAÍS https://t.co/TzXLNtkzMR","""Los pilotos de Ryanair desconvocan la huelga tras ver reconocidos sus sindicatos"" by El País via Últimas noticias | EL PAÍS https://t.co/TzXLNtkzMR"
9.270000e+17,positive,True,0,0,"@Iberia @lavecinarubia Si ,por favor las declaraciones de amor entre los #rubijarena no pueden quedarse en una ma… https://t.co/GWKJGhhubY",NaN,43043.71181,NaN,NaN,9.270000e+17,"@Iberia @lavecinarubia Si ,por favor las declaraciones de amor entre los #rubijarena no pueden quedarse en una maleta . #cuandooigoqueJonledicevecinita ??","@Iberia @lavecinarubia Si ,por favor las declaraciones de amor entre los #rubijarena no pueden quedarse en una maleta . #cuandooigoqueJonledicevecinita ??"
9.370000e+17,neutral,True,0,0,@Iberia Me dirías por favor que costo tiene?,NaN,43071.64167,NaN,NaN,9.370000e+17,@Iberia Me dirías por favor que costo tiene?,@Iberia Me dirías por favor que costo tiene?
9.440000e+17,negative,True,0,0,"@SupermanlopezN @Iberia @giroditalia Champion, no vuele más con esos descarados.",NaN,43090.97014,NaN,Central Time (US & Canada),9.440000e+17,"@SupermanlopezN @Iberia @giroditalia Champion, no vuele más con esos descarados.","@SupermanlopezN @Iberia @giroditalia Champion, no vuele más con esos descarados."
9.380000e+17,negative,True,0,0,@SrtaFarrellDM @KLM @Iberia Eso de avianca es verdad a mi mama le paso dos veces terrible esa aerolínea ?,NaN,43075.03056,NaN,Eastern Time (US & Canada),9.380000e+17,@SrtaFarrellDM @KLM @Iberia Eso de avianca es verdad a mi mama le paso dos veces terrible esa aerolínea ?,@SrtaFarrellDM @KLM @Iberia Eso de avianca es verdad a mi mama le paso dos veces terrible esa aerolínea ?
